# `MEDUSA`
aka. Dynamic-Prospective LCA aka. Union(premise, temporalis)

In [1]:
from bw_temporalis import easy_timedelta_distribution, easy_datetime_distribution, TemporalisLCA, Timeline, TemporalDistribution
from bw_temporalis.lcia import characterize_methane, characterize_co2
import bw2data as bd
import bw2calc as bc
import bw_graph_tools as graph
import numpy as np
import pandas as pd

In [2]:
bd.projects.set_current('default')
bd.projects

Brightway2 projects manager with 16 objects:
	LCA_VL_masterdb
	Test
	as23_project
	as23_project_bw2
	default
	ei39
	ei39_premise_scenarios
	medusa_test2
	medusa_test3
	premise-example
	premise_ei39
	test
	tictac
	tictacthree
	📊📈💎🕤🗓️
	🪒
Use `projects.report()` to get a report on all projects.

In [3]:
try: 
    bd.projects.delete_project('medusa_test1')
    bd.projects.purge_deleted_directories()
except:
    pass

bd.projects.set_current("medusa_test3")

### Create example databases

In [4]:
bd.Database('temporalis-bio').write({
    ('temporalis-bio', "CO2"): {
        "type": "emission",
        "name": "carbon dioxide",
        "temporalis code": "co2",
    },
    ('temporalis-bio', "CH4"): {
        "type": "emission",
        "name": "methane",
        "temporalis code": "ch4",
    },
})

bd.Database('background_2023').write({
    ('background_2023', 'electricity_mix'): {
        'name': 'Electricity mix',
        'exchanges': [
            {
                'amount': 1,
                'type': 'production',
                'input': ('background_2023', 'electricity_mix'),
            },
            {
                'amount': 1,
                'type': 'technosphere',
                'input': ('background_2023', 'electricity_wind'),
            },
        ]
    },
    ('background_2023', 'electricity_wind'): {
        'name': 'Electricity production, wind',
        'exchanges': [
            {
                'amount': 1,
                'type': 'production',
                'input': ('background_2023', 'electricity_wind'),
            },
            {
                'amount': 1,
                'type': 'biosphere',
                'input': ('temporalis-bio', 'CO2'),
            },
        ]
    }
})

bd.Database('background_2020').write({
    ('background_2020', 'electricity_mix'): {
        'name': 'Electricity mix',
        'exchanges': [
            {
                'amount': 1,
                'type': 'production',
                'input': ('background_2020', 'electricity_mix'),
            },
            {
                'amount': 1,
                'type': 'technosphere',
                'input': ('background_2020', 'electricity_wind'),
            },
        ]
    },
    ('background_2020', 'electricity_wind'): {
        'name': 'Electricity production, wind',
        'exchanges': [
            {
                'amount': 1,
                'type': 'production',
                'input': ('background_2020', 'electricity_wind'),
            },
            {
                'amount': 468745,
                'type': 'biosphere',
                'input': ('temporalis-bio', 'CO2'),
            },
        ]
    }
})

bd.Database('foreground').write({
    ('foreground', 'heat_from_hydrogen'): {
        'name': 'Heat production, hydrogen',
        'exchanges': [
            {
                'amount': 1,
                'type': 'production',
                'input': ('foreground', 'heat_from_hydrogen'),
            },
            {
                'amount': 1,
                'type': 'technosphere',
                'input': ('foreground', 'electrolysis'),
                'temporal_distribution': # e.g. because some hydrogen was stored in the meantime
                    easy_timedelta_distribution(
                    start=-3,
                    end=0, # Range includes both start and end
                    resolution="Y",  # M for months, Y for years, etc.
                    steps=2,
                ),
            },
        ]
    },('foreground', 'electrolysis'): {
        'name': 'Hydrogen production, electrolysis',
        'exchanges': [
            {
                'amount': 1,
                'type': 'production',
                'input': ('foreground', 'electrolysis'),
            },
            {
                'amount': 1,
                'type': 'technosphere',
                'input': ('background_2023', 'electricity_mix'),
            },
        ]
    },
})

100%|██████████| 2/2 [00:00<?, ?it/s]

Vacuuming database 
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 2/2 [00:00<?, ?it/s]

Vacuuming database 


Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 2/2 [00:00<?, ?it/s]


Vacuuming database 
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 2/2 [00:00<?, ?it/s]


Vacuuming database 


In [5]:
bd.Method(("GWP", "example")).write([
    (('temporalis-bio', "CO2"), 1),
    (('temporalis-bio', "CH4"), 25),
])

In [6]:
demand = {('foreground', 'heat_from_hydrogen'): 1}
gwp = ('GWP', 'example')

# Static LCA

In [7]:
slca = bc.LCA(demand, gwp)
slca.lci()
slca.lcia()
slca.score

1.0

# Custom `EdgeExtractor` class

In [8]:
from dataclasses import dataclass
from heapq import heappop, heappush
from typing import Callable, List

from bw_temporalis import TemporalisLCA, TemporalDistribution


@dataclass
class Edge:
    """
    Class for storing a temporal edge with source and target.

    Leaf edges link to a source process which is a leaf in
    our graph traversal (either through cutoff or a filter
    function).

    Attributes
    ----------
    distribution : TemporalDistribution
    leaf : bool
    consumer : int
    producer : int

    """

    distribution: TemporalDistribution
    leaf: bool
    consumer: int
    producer: int


class EdgeExtracter(TemporalisLCA):
    def __init__(self, *args, edge_filter_function: Callable = None, **kwargs):
        super().__init__(*args, **kwargs)
        if edge_filter_function:
            self.edge_ff = edge_filter_function
        else:
            self.edge_ff = lambda x: False

    def build_edge_timeline(self, node_timeline: bool | None = False) -> List:
        heap = []
        timeline = []

        for edge in self.edge_mapping[self.unique_id]:
            node = self.nodes[edge.producer_unique_id]
            heappush(
                heap,
                (
                    1 / node.cumulative_score,
                    self.t0 * edge.amount,
                    node,
                ),
            )
            timeline.append(
                Edge(
                    distribution=self.t0 * edge.amount,
                    leaf=False,
                    consumer=self.unique_id,
                    producer=node.activity_datapackage_id,
                )
            )

        while heap:
            _, td, node = heappop(heap)

            for edge in self.edge_mapping[node.unique_id]:
                row_id = self.nodes[edge.producer_unique_id].activity_datapackage_id
                col_id = node.activity_datapackage_id
                exchange = self.get_technosphere_exchange(
                    input_id=row_id,
                    output_id=col_id,
                )
                value = (
                    self._exchange_value(
                        exchange=exchange,
                        row_id=row_id,
                        col_id=col_id,
                        matrix_label="technosphere_matrix",
                    )
                    / node.reference_product_production_amount
                )
                producer = self.nodes[edge.producer_unique_id]
                leaf = self.edge_ff(row_id)

                distribution = (td * value).simplify()
                timeline.append(
                    Edge(
                        distribution=distribution,
                        leaf=leaf,
                        consumer=node.activity_datapackage_id,
                        producer=producer.activity_datapackage_id,
                    )
                )
                if not leaf:
                    heappush(
                        heap,
                        (
                            1 / node.cumulative_score,
                            distribution,
                            producer,
                        ),
                    )
        return timeline

In [9]:
SKIPPABLE = [node.id for node in bd.Database('background_2020')] + [
    node.id for node in bd.Database('background_2023')
]

def filter_function(database_id: int) -> bool:
    return database_id in SKIPPABLE

# Example

In [10]:
eelca = EdgeExtracter(slca, edge_filter_function=filter_function)

Starting graph traversal
Calculation count: 3


In [11]:
timeline = eelca.build_edge_timeline()

In [12]:
from medusa_tools import create_grouped_edge_dataframe
from datetime import datetime

dates_list = [
        datetime.strptime("2020", "%Y"),
        datetime.strptime("2023", "%Y"),
    ] # we probably should comebine this with database_date_dict, or include the time data in the databases directly
timeline_df = create_grouped_edge_dataframe(timeline, dates_list, interpolation_type="linear")

In [13]:
timeline_df
# if leaf -> include prospective db in dp???

,year,producer,consumer,amount,date,interpolation_weights,producer_name,consumer_name
0,2020,35,40,0.5,2020-01-01,{2020: 1},Electricity mix,"Hydrogen production, electrolysis"
1,2020,40,39,0.5,2020-01-01,{2020: 1},"Hydrogen production, electrolysis","Heat production, hydrogen"
2,2023,35,40,0.5,2023-01-01,{2023: 1},Electricity mix,"Hydrogen production, electrolysis"
3,2023,39,-1,1.0,2023-01-01,{2023: 1},"Heat production, hydrogen",-1
4,2023,40,39,0.5,2023-01-01,{2023: 1},"Hydrogen production, electrolysis","Heat production, hydrogen"


In [14]:
from medusa_tools import create_datapackage_from_edge_timeline

database_date_dict = {
            2020: 'background_2020',
            2023: 'background_2023',
        }
dp = create_datapackage_from_edge_timeline(timeline_df, database_date_dict)

Current row: 2020  |  Electricity mix  |  Hydrogen production, electrolysis
Row links to background database
New link goes to background_2020 for year 2020
Previous producer: ('background_2023', 'electricity_mix'), id = 35
Previous consumer: ('foreground', 'electrolysis'), id = 40
New producer id = 37
New consumer id = 40002020

Current row: 2020  |  Hydrogen production, electrolysis  |  Heat production, hydrogen
Row contains internal foreground edge - exploding to new time-specific nodes
New producer id = 40002020
New consumer id = 39002020

Current row: 2023  |  Electricity mix  |  Hydrogen production, electrolysis
Row links to background database
New link goes to background_2023 for year 2023
Previous producer: ('background_2023', 'electricity_mix'), id = 35
Previous consumer: ('foreground', 'electrolysis'), id = 40
New producer id = 35
New consumer id = 40002023

Current row: 2023  |  Heat production, hydrogen  |  -1
Row contains the functional unit - exploding to new time-specific

In [15]:
from medusa_tools import prepare_medusa_lca_inputs

# fu, data_objs, remapping = bd.prepare_lca_inputs(demand=demand, method=gwp)

# using this instead of bd.prepare_lca_inputs() should now include all databases (incl. prospective ones) in data_objs
fu, data_objs, remapping = prepare_medusa_lca_inputs(demand=demand, method=gwp) 


In [16]:
# functional unit still links to original temporalis db, not the newly created node. 
fu = {key*1000000+2023: val for key, val in fu.items()}
fu

{39002023: 1}

In [17]:
lca = bc.LCA(fu, data_objs = data_objs + [dp], remapping_dicts=remapping)

In [18]:
lca.load_lci_data(nonsquare_ok=True)
acts = set([v for v in lca.dicts.activity.reversed.values()])
prods = set([v for v in lca.dicts.product.reversed.values()])
prods-acts
[bd.get_node(id=missing_product).key for missing_product in prods-acts]

[]

In [19]:
lca.lci()
lca.lcia()
print('new lca score:', lca.score)
print('old lca score:', slca.score)

new lca score: 0.25
old lca score: 1.0


In [20]:
# The results dont make sense. Also, they dont dont change when changing the amount of co2 emittet in the background_2020 db